### jupyter notebook to do feature engineering

author: Jan Jörg
date: 14.03.2024

In [37]:
import json
from db import get_database
import pandas as pd
import numpy as np

In [38]:
# import authentication data

with open('./infos.json') as f:
    infos = json.load(f)
    onet = infos['onet']
    onetUsername = onet['username']
    onetPassword = onet['password']
    mongodb = infos['mongodb']
    mongoUusername = mongodb['username']
    mongoPpassword = mongodb['password']
    mongoUrl = mongodb['connectionString']

In [39]:

dbname = get_database()

collection = dbname["joined"]
fo_col = dbname["fo_joined"]
job_zone_col = dbname["job_zone"]
abilities_col = dbname["abilities"]
skills_col = dbname["skills"]
education_col = dbname["education"]

documents = collection.find()
fo = fo_col.find()
job_zone = job_zone_col.find()
abilities = abilities_col.find()
skills = skills_col.find()
education = education_col.find()

df = pd.DataFrame(list(documents))
fo_df = pd.DataFrame(list(fo))
job_zone_df = pd.DataFrame(list(job_zone))
abilities_df = pd.DataFrame(list(abilities))
skills_df = pd.DataFrame(list(skills))
education_df = pd.DataFrame(list(education))

In [40]:
# Remove empty cols
print("Total documents: ", len(df))

# Drop columns where some values are null
df = df.dropna(how='all', axis=1)

# Drop rows where all some are null
df = df.dropna(how='all', axis=0)

# Replace any remaining NaN values with 0
df = df.fillna(0)

print("Total documents after dropping na: ", len(df))

Total documents:  846
Total documents after dropping na:  846


In [42]:
# Remove columns that are not useful
df = df.drop(['additional_information',"tasks","related_occupations","display","technology_skills","tools_technology","_id","tools_used","work_context","interests","work_values","knowledge","detailed_work_activities","work_activities","work_styles"], axis=1)
df.head(2)

,occupation,skills,abilities,job_zone,education,isco08,Name_de
0,"{'code': '27-2011.00', 'title': 'Actors', 'tag...","{'element': [{'id': '2.A.1.a', 'related': 'htt...","{'element': [{'id': '1.A.1.a.3', 'related': 'h...","{'value': 2, 'title': 'Job Zone Two: Some Prep...",{'level_required': {'category': [{'name': 'Les...,2655,Schauspieler
1,"{'code': '23-1021.00', 'title': 'Administrativ...","{'element': [{'id': '2.A.1.b', 'related': 'htt...","{'element': [{'id': '1.A.1.b.5', 'related': 'h...","{'value': 5, 'title': 'Job Zone Five: Extensiv...",{'level_required': {'category': [{'name': 'Doc...,2612,Richter


In [43]:
occupation = pd.DataFrame(list(df["occupation"])) 

# List of columns to drop
cols_to_drop = ["code","updated","sample_of_reported_job_titles","summary_resources","details_resources","custom_resources", "tags"]

# Drop the columns
occupation = occupation.drop(cols_to_drop, axis=1)

occupation = occupation.iloc[:, 0]
df["occupation"] = occupation

df["occupation"].head(3)

0                                               Actors
1    Administrative Law Judges, Adjudicators, and H...
2    Aerospace Engineering and Operations Technolog...
Name: occupation, dtype: object

In [44]:
# Initialize all skill columns to 0
for _, skill in skills_df.iterrows():
    skilltext = ("s" + str(skill["skill_id"]))
    df[skilltext] = 0

# add Value to each skill
for index, row in df["skills"].items():
    row = pd.DataFrame(row['element'])

    # Merge the skills dataframe with the skills_df dataframe to get the new skill id   
    row = row.merge(skills_df, left_on='id', right_on='id', how='left')

    for _, skill_row in row.iterrows():
        dftext = ("s" + str(skill_row["skill_id"]))
        score_value = skill_row["score"]['value']
        df.loc[index, dftext] = score_value

df.head(2)

,occupation,skills,abilities,job_zone,education,isco08,Name_de,s10,s1,s4,...,s29,s33,s8,s9,s14,s17,s20,s26,s27,s34
0,Actors,"{'element': [{'id': '2.A.1.a', 'related': 'htt...","{'element': [{'id': '1.A.1.a.3', 'related': 'h...","{'value': 2, 'title': 'Job Zone Two: Some Prep...",{'level_required': {'category': [{'name': 'Les...,2655,Schauspieler,47,72,69,...,0,0,47,47,41,35,25,0,0,0
1,"Administrative Law Judges, Adjudicators, and H...","{'element': [{'id': '2.A.1.b', 'related': 'htt...","{'element': [{'id': '1.A.1.b.5', 'related': 'h...","{'value': 5, 'title': 'Job Zone Five: Extensiv...",{'level_required': {'category': [{'name': 'Doc...,2612,Richter,78,81,72,...,16,3,72,50,69,41,44,0,0,0


In [45]:
# Initialize all ability columns to 0
for _, ability in abilities_df.iterrows():
    abilitytext = ("a" + str(ability["ability_id"]))
    df[abilitytext] = 0

# add Value to each ability
for index, row in df["abilities"].items():
    row = pd.DataFrame(row['element'])

    # Merge the skills dataframe with the skills_df dataframe to get the new skill id   
    row = row.merge(abilities_df, left_on='id', right_on='id', how='left')

    for _, ability_row in row.iterrows():
        dftext = ("a" + str(ability_row["ability_id"]))
        score_value = ability_row["score"]['value']
        df.loc[index, dftext] = score_value

df.head(2)

,occupation,skills,abilities,job_zone,education,isco08,Name_de,s10,s1,s4,...,a33,a34,a36,a39,a41,a43,a44,a28,a40,a49
0,Actors,"{'element': [{'id': '2.A.1.a', 'related': 'htt...","{'element': [{'id': '1.A.1.a.3', 'related': 'h...","{'value': 2, 'title': 'Job Zone Two: Some Prep...",{'level_required': {'category': [{'name': 'Les...,2655,Schauspieler,47,72,69,...,25,25,22,3,0,0,0,25,3,0
1,"Administrative Law Judges, Adjudicators, and H...","{'element': [{'id': '2.A.1.b', 'related': 'htt...","{'element': [{'id': '1.A.1.b.5', 'related': 'h...","{'value': 5, 'title': 'Job Zone Five: Extensiv...",{'level_required': {'category': [{'name': 'Doc...,2612,Richter,78,81,72,...,0,0,0,28,16,0,31,22,6,0


In [46]:
new_education = list()

for index, row in df["education"].items():
    row = pd.DataFrame(row['level_required']['category'])

    # Check if 'score' exists
    if 'score' in row.columns:
        # Extract 'value' from 'score' and convert it to a DataFrame column
        max_value = row['score'].apply(lambda d: d['value'])

        # Find the index of the row with the highest score
        max_score_index = max_value.idxmax()

        # Select only the row with the highest score
        row = row.loc[[max_score_index]]

    cols_to_drop = ["description", "score"]
    row = row.drop(cols_to_drop, axis=1, errors='ignore')

   
    # Merge the education dataframe with the education_df dataframe to get the new education level
    row = row.merge(education_df, left_on='name', right_on='name', how='left')

    cols_to_drop = ["id", "_id", "name"]
    row = row.drop(cols_to_drop, axis=1, errors='ignore')

    # Convert DataFrame to numpy array
    row = row.to_numpy().flatten()[0]

    new_education.append(row)


df["education"] = new_education

df.head(2)

,occupation,skills,abilities,job_zone,education,isco08,Name_de,s10,s1,s4,...,a33,a34,a36,a39,a41,a43,a44,a28,a40,a49
0,Actors,"{'element': [{'id': '2.A.1.a', 'related': 'htt...","{'element': [{'id': '1.A.1.a.3', 'related': 'h...","{'value': 2, 'title': 'Job Zone Two: Some Prep...",1,2655,Schauspieler,47,72,69,...,25,25,22,3,0,0,0,25,3,0
1,"Administrative Law Judges, Adjudicators, and H...","{'element': [{'id': '2.A.1.b', 'related': 'htt...","{'element': [{'id': '1.A.1.b.5', 'related': 'h...","{'value': 5, 'title': 'Job Zone Five: Extensiv...",9,2612,Richter,78,81,72,...,0,0,0,28,16,0,31,22,6,0


In [47]:
new_job_zones = list()

for index, row in df["job_zone"].items():
    value = row["value"]
    
    row = job_zone_df[job_zone_df["value"] == value]
  
    cols_to_drop = ["id", "_id", "related_experience", "education", "title", "job_training","job_zone_examples","value"]
    row = row.drop(cols_to_drop, axis=1, errors='ignore')

    # Convert DataFrame to numpy array
    row = row.to_numpy().flatten()[0]
    
    new_job_zones.append(row)

df["job_zone"] = new_job_zones

df.head(2)

,occupation,skills,abilities,job_zone,education,isco08,Name_de,s10,s1,s4,...,a33,a34,a36,a39,a41,a43,a44,a28,a40,a49
0,Actors,"{'element': [{'id': '2.A.1.a', 'related': 'htt...","{'element': [{'id': '1.A.1.a.3', 'related': 'h...",1,1,2655,Schauspieler,47,72,69,...,25,25,22,3,0,0,0,25,3,0
1,"Administrative Law Judges, Adjudicators, and H...","{'element': [{'id': '2.A.1.b', 'related': 'htt...","{'element': [{'id': '1.A.1.b.5', 'related': 'h...",4,9,2612,Richter,78,81,72,...,0,0,0,28,16,0,31,22,6,0


In [48]:
def convert_np_array_to_list(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

In [49]:
# Remove columns that are not useful
df = df.drop(["skills","abilities","occupation"], axis=1)
df.head(2)

,job_zone,education,isco08,Name_de,s10,s1,s4,s11,s22,s6,...,a33,a34,a36,a39,a41,a43,a44,a28,a40,a49
0,1,1,2655,Schauspieler,47,72,69,47,13,50,...,25,25,22,3,0,0,0,25,3,0
1,4,9,2612,Richter,78,81,72,78,16,66,...,0,0,0,28,16,0,31,22,6,0


In [50]:
fo_df = pd.merge(df, fo_df, left_on='isco08', right_on="isco08", how='left')
fo_df = fo_df.drop_duplicates(subset=['isco08'])
fo_df = fo_df.dropna()

In [51]:
dbname = get_database()

insert_df = df.applymap(convert_np_array_to_list)
insert_fo_df = fo_df.applymap(convert_np_array_to_list)

collection_name = dbname["with_id"]
fo_col_name = dbname["fo_with_id"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(insert_df.to_dict("records"))
fo_col_name.insert_many(insert_fo_df.to_dict("records"))

/var/folders/ms/0wqr6tr506lfp142wmz0qph40000gn/T/ipykernel_9103/3673509860.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  insert_df = df.applymap(convert_np_array_to_list)
/var/folders/ms/0wqr6tr506lfp142wmz0qph40000gn/T/ipykernel_9103/3673509860.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  insert_fo_df = fo_df.applymap(convert_np_array_to_list)


InsertManyResult([ObjectId('663e1dd1db2fdc9e602a8fba'), ObjectId('663e1dd1db2fdc9e602a8fc7'), ObjectId('663e1dd1db2fdc9e602a8f8d'), ObjectId('663e1dd1db2fdc9e602a8f21'), ObjectId('663e1dd1db2fdc9e602a8ee6'), ObjectId('663e1dd1db2fdc9e602a8f4b'), ObjectId('663e1dd1db2fdc9e602a9104'), ObjectId('663e1dd1db2fdc9e602a9105'), ObjectId('663e1dd1db2fdc9e602a908b'), ObjectId('663e1dd1db2fdc9e602a8f6d'), ObjectId('663e1dd1db2fdc9e602a8fcd'), ObjectId('663e1dd1db2fdc9e602a9101'), ObjectId('663e1dd1db2fdc9e602a9035'), ObjectId('663e1dd1db2fdc9e602a8fc9'), ObjectId('663e1dd1db2fdc9e602a8f0a'), ObjectId('663e1dd1db2fdc9e602a8ea5'), ObjectId('663e1dd1db2fdc9e602a9033'), ObjectId('663e1dd1db2fdc9e602a8f72'), ObjectId('663e1dd1db2fdc9e602a8f12'), ObjectId('663e1dd1db2fdc9e602a8f09'), ObjectId('663e1dd1db2fdc9e602a90fd'), ObjectId('663e1dd1db2fdc9e602a90fe'), ObjectId('663e1dd1db2fdc9e602a90ff'), ObjectId('663e1dd1db2fdc9e602a9100'), ObjectId('663e1dd1db2fdc9e602a9102'), ObjectId('663e1dd1db2fdc9e602a8e